<a href="https://colab.research.google.com/github/hyemworks/make-your-firstGAN-with-pytorch/blob/main/ch04_cuda_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHAPTER 04 &nbsp;&nbsp;&nbsp;&nbsp;CUDA Basics**

## **Numpy VS Python**


In [2]:
import torch
import numpy as np


In [12]:
# 정사각행렬의 크기
size = 600

a = np.random.rand(size,size)
b = np.random.rand(size,size) # 0과 1사이의 임의의 값으로 두 넘파이 행렬을 채움.

In [3]:
%%timeit
# 한 셀에서 시간이 얼마나 오래 걸리는지 표시해줌.

# 01. numpy를 사용하는 경우 - 12.1 ms per loop
x = np.dot(a,b)

100 loops, best of 5: 12.1 ms per loop


In [4]:
%%timeit

# 02. numpy를 사용하지 않는 경우 - 2min 47s per loop
c = np.zeros((size,size))

for i in range(size):
  for j in range(size):
    for k in range(size):
      c[i,j] += a[i,k]*b[k,j]

1 loop, best of 5: 2min 47s per loop


↪ 간단히 비교해보면, Numpy가 순수 Python보다 **행렬곱 계산**이 훨씬 빠르다는 것을 알 수 있다!

## **엔비디아 CUDA**

✅ **NVIDIA, 엔비디아**
  - GPU 시장의 리더역할을 하고 있는 기업
  - 강력한 하드웨어 가속 기능을 갖춘, 머신러닝 연구에 표준적인 소프트웨어 툴을 제공함.<br>
  <br>
  🦋 <b>CUDA</b>$^{Compute-Unified-Device-Architecture}$<br>
  &nbsp;&nbsp;&nbsp;&nbsp;: 엔비디아에서 제공하는 소프트웨어 프레임 워크
  <br>
  &nbsp;&nbsp;&nbsp;&nbsp;▶ 단점 <br>
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: 엔비디아의 GPU에서만 동작.<br>
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cf) 경쟁사인 AMD는 최근에서야 필적할 만한 프레임워크를 개발함. 

## **CUDA를 Python에서 사용하기**

: Colab 노트북 내 설정방법<br>
↪ 상단 메뉴 > [Runtime] > [Change Runtime Type] > [Hardware Accelerator] > **GPU**

In [6]:
# tensor 만들기

x = torch.FloatTensor([3.5])
print(x.type()) # 데이터 타입 체크

torch.FloatTensor


In [9]:
# GPU에서 tensor 만들기

x = torch.cuda.FloatTensor([3.5])
print(x.type()) # 데이터 타입 체크
print(x.device) # 어떤 장치에 텐서가 올라가 있는지 체크
x.device  

torch.cuda.FloatTensor
cuda:0


device(type='cuda', index=0)

In [10]:
# GPU를 통해 텐서 계산

y = x*x
y

tensor([12.2500], device='cuda:0')

<br>

GPU를 사용함으로써 이득을 얻으려면, 정말 많은 데이터를 각 GPU 코어에 나눠서 투입해야함.<br>
➡ 앞에서 했던 행렬곱을 GPU에서 해보자

In [13]:
aa = torch.cuda.FloatTensor(a)
bb = torch.cuda.FloatTensor(b)

In [14]:
%%timeit

# torch.matmul => pytorch에서 "행렬곱"을 하는 명령어

cc = torch.matmul(aa,bb)

The slowest run took 480.18 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 116 µs per loop


<br>
<br>

아래 셀 == 파이토치의 device에 대한 표준적인 코드처리<br>
▶ **GPU**를 기본값으로 설정하고, **CUDA가 사용 가능하지 않을 때**에만 **CPU**를 이용함!

In [15]:
## Check if CUDA is available
## if yes, set default tensor type to cuda

if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda : ", torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

using cuda :  Tesla T4


device(type='cuda')

---

## **Summary**
>✅ **GPU$^{Graphics-Processing-Unit}$, 그래픽 처리 장치** <br>
: 많은 수의 연산 코어를 통해 특정한 작업을 빠르게 할 수 있음.<br>
&nbsp;&nbsp;&nbsp;&nbsp; ↪ **병렬로 행렬곱을 처리**하는 것 등을 포함하여, 단순 계산에 특화됨.<br>
: 그래픽 관련 연산외에도 머신러닝 관련 계산이 빠름.
<br>
<br>
✅ **CUDA$^{Compute-Unified-Device-Architecture}$** <br>
: NVIDIA의 향상된 연산용 프로그래밍 프레임워크.<br>

> GPU는 단일계산에 대해서는 CPU보다 느릴 수 있음.<br>
따라서 데이터가 충분히 크지 않다면 GPU를 사용하는 이득이 별로 없을 수 있음.

---